In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns

In [ ]:
'''
    This Kernel is an attempt to cross-validate the other ML network results, maybe this can be tuned a bit to improve 
    the result but wanted to move onto other challenges. 
'''

In [2]:
df=pd.read_csv('Clean_Data/relearn.csv')

In [3]:
df.index=df['PassengerId']
df.drop('PassengerId',axis=1,inplace=True)

In [4]:
x=df.drop('Survived',axis=1)
y=df['Survived']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.20,random_state=101)

In [7]:
import tensorflow as tf

C:\Users\Adam\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [8]:
feat_cols=[]

for col in x.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [9]:
input_func=tf.estimator.inputs.pandas_input_fn(x=x_train,y=y_train,batch_size=100,num_epochs=300,shuffle=True)

In [10]:
Classifier=tf.estimator.DNNClassifier(hidden_units=[400,200,100,50],n_classes=2,feature_columns=feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Adam\\AppData\\Local\\Temp\\tmpxbccxevq', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000000000E151BE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [11]:
Classifier.train(input_fn=input_func,steps=1000)

Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Adam\AppData\Local\Temp\tmpxbccxevq\model.ckpt.
INFO:tensorflow:loss = 69.598595, step = 1
INFO:tensorflow:global_step/sec: 262.452
INFO:tensorflow:loss = 49.81546, step = 101 (0.381 sec)
INFO:tensorflow:global_step/sec: 353.337
INFO:tensorflow:loss = 44.474873, step = 201 (0.283 sec)
INFO:tensorflow:global_step/sec: 338.964
INFO:tensorflow:loss = 40.645187, step = 301 (0.296 sec)
INFO:tensorflow:global_step/sec: 341.277
INFO:tensorflow:loss = 36.009407, 

In [12]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=x_test,batch_size=len(x_test),shuffle=False)


In [13]:
note_predictions=list(Classifier.predict(input_fn=pred_fn))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Adam\AppData\Local\Temp\tmpxbccxevq\model.ckpt-516
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [14]:
final_preds=[]

for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [15]:
from sklearn.metrics import classification_report,confusion_matrix

In [16]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          0       0.79      0.92      0.85        24
          1       0.87      0.68      0.76        19

avg / total       0.82      0.81      0.81        43



In [18]:
df2=pd.read_csv('Clean_Data/test_relearn.csv')

In [19]:
df2.index=df2['PassengerId']
df2.drop('PassengerId',axis=1,inplace=True)

In [20]:
pred_fn2=tf.estimator.inputs.pandas_input_fn(x=df2,batch_size=len(df2),shuffle=False)

In [21]:
note_predictions2=list(Classifier.predict(input_fn=pred_fn2))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Adam\AppData\Local\Temp\tmpxbccxevq\model.ckpt-516
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [22]:
final_preds2=[]

for pred in note_predictions2:
    final_preds2.append(pred['class_ids'][0])

In [23]:
submission=pd.DataFrame(index=df2.index)
submission['Survived']=list(final_preds2)

In [24]:
submission.to_csv('titanic_submission.csv', encoding='utf-8', index=True)